In [1]:
cd ~/git/thinking_like_transformers

/Users/ellenar/git/thinking_like_transformers


In [2]:
#@title Imports
import jax
import numpy as np

# The default of float16 can lead to discrepancies between outputs of
# the compiled model and the RASP program.
jax.config.update('jax_default_matmul_precision', 'float32')

from tracr_utils.lib.tracr.compiler import compiling, assemble
from tracr.rasp import rasp
import torch
import torch.nn as nn

In [3]:
from model.transformer_model import TransformerModel
from inference.algorithms import make_length, reverse, hist
from inference.load_weights import extract_config, build_state_dict
from utils import register_hooks

In [46]:
input_= ['bos',3,2,2,2]
algorithm = hist()
bos = "bos"
compiled_model = compiling.compile_rasp_to_model(
    algorithm,
    vocab={1,2,3,4},
    max_seq_len=5,
    compiler_bos=bos,
)
decoded_output_1 = compiled_model.apply(input_)

In [47]:
model_config = extract_config(compiled_model)
state_dict = build_state_dict(compiled_model)
torch_model = TransformerModel(model_config).eval()
activations_dict = {}
register_hooks(torch_model, activations_dict)
torch_model.load_state_dict(state_dict)
   
x = compiled_model.input_encoder.encode(input_)
logits = torch_model(x)
# decode logits
max_output_indices = torch.argmax(logits, dim=1)
decoded_output_2 = compiled_model.output_encoder.decode(max_output_indices.tolist())[1:]

In [60]:
logits

tensor([[ 0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
          1.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
          0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
          0.0000e+00,  0.0000e+00,  1.0000e+00,  0.0000e+00],
        [ 1.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
          1.0000e+00,  0.0000e+00,  1.0000e+00,  0.0000e+00,  0.0000e+00,
         -1.9073e-06,  1.9073e-06,  5.0000e-01,  0.0000e+00,  0.0000e+00,
          1.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00],
        [ 0.0000e+00,  1.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
          1.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  1.0000e+00,
          0.0000e+00,  0.0000e+00,  2.5000e-01,  0.0000e+00,  1.0000e+00,
          0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00],
        [ 0.0000e+00,  0.0000e+00,  1.0000e+00,  0.0000e+00,  0.0000e+00,
          1.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  1.0000e+00,


In [77]:
output = []
for i in range(logits.shape[0]):
    out=1/torch.round(logits[i,:], decimals=3)
    out=int(torch.max(torch.nan_to_num(out, nan=0.0, posinf=0.0, neginf=0.0)).item()-1)
    output.append(out)